<a href="https://colab.research.google.com/github/kgpark88/deeplearning/blob/master/model_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hugging Face에서 제공하는 Pipeline API를 활용해서 ooo 서비스를 구현하세요.
## ooo : 영화리뷰 감성분류, 개체명분류, QA(Question Answering)

### transformers 패키지 설치

In [49]:
!pip install transformers

### 라이브러리 임포트

In [50]:
from transformers import ElectraTokenizer, ElectraForQuestionAnswering, pipeline
from pprint import pprint

### Hugging Face 파이프라인 사용

In [52]:
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-v2-distilled-korquad-384")
model = ElectraForQuestionAnswering.from_pretrained("monologg/koelectra-small-v2-distilled-korquad-384")

qa = pipeline("question-answering", tokenizer=tokenizer, model=model)

### ooo 서비스 실행
### ooo : 영화리뷰 감성분류, 개체명분류, QA(Question Answering)

In [40]:
question = "유가 상승 이유는?"
# question = "텍사스산 원유 거래가격?"

In [41]:
context = "국제 유가가 미국의 한파에 따른 원유 생산 셧다운 여파로 상승세를 이어갔다. 17일(현지시간) 뉴욕상업거래소(NYMEX)에서 3월 인도분 서부 텍사스산 원유는 전날보다 배럴당 1.8%(1.09달러) 오른 61.14달러에 거래를 마쳤다. 런던 ICE선물거래소의 4월물 브렌트유는 오후 3시45분 기준 배럴당 1.7%(1.08달러) 상승한 64.43달러에 거래되고 있다.기록적인 한파와 폭설로 미국 텍사스주 일대가 정전되면서 원유·정유 관련 시설들이 문을 닫거나 가동을 줄이면서 공급난을 초래했다."

In [53]:
result = qa({"question": question, "context": context})
print(result)

{'score': 0.4284178614616394, 'start': 18, 'end': 27, 'answer': '원유 생산 셧다운'}


### 답변(answer)만 출력하세요.

In [54]:
answer = result["answer"]
print(answer)

원유 생산 셧다운
